In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.client import device_lib
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow
import numpy as np

In [3]:
f = open('HerMajestySpeechesDataset/train.txt', 'r', encoding='utf-8')
# text = f.read()
texts = f.readlines()
f.close()

In [3]:
len(texts)

2361

In [4]:
tokenizer = Tokenizer(oov_token='<unk>')
tokenizer.fit_on_texts(texts) 

texts2ids = tokenizer.texts_to_sequences(texts)
print ("Texts as IDs:", texts2ids[:10])
ids2texts = tokenizer.sequences_to_texts(texts2ids)
print ("IDs back to texts:", ids2texts[:10])

Texts as IDs: [[2, 181, 999, 5, 39, 584, 41, 2094, 2095, 21, 1165, 530], [21, 9, 163, 246, 8, 1000, 25, 424, 1166, 5, 18, 20], [68, 195, 781], [2, 355, 3, 7, 1347, 722, 53, 585, 40, 213, 2, 297, 3, 531, 208, 1001], [2, 181, 68, 195, 6, 781, 2096, 21, 532, 117, 872, 4, 247], [2, 195, 49, 2097, 6, 2951, 1167, 6, 151, 224, 196, 491, 533, 2098, 197, 21, 7, 2952, 2953], [8, 13, 1168, 205, 873, 5, 114, 10, 455, 187, 298, 586, 2, 259, 11, 111, 205], [58, 3, 2, 455, 10, 29, 50, 649, 20, 10, 79, 11, 66, 12, 2, 312, 3, 68], [17, 271, 7, 59, 83, 8, 782, 5, 456, 2099, 2, 2954, 3, 2, 356, 4, 214, 10, 874, 49, 492, 5, 237, 129, 4, 493, 5, 7, 875, 42], [2, 355, 3, 783, 2100, 2955, 16, 457, 5, 20, 43, 492, 248, 4, 91, 2, 69, 2956, 11, 141]]
IDs back to texts: ["the queen's messages to those celebrating their 90th birthdays on 21 april", 'on this shared occasion i send my warm congratulations to you all', 'christmas broadcast 2006', 'the birth of a baby brings great happiness but then the business of g

In [66]:
print(len(texts2ids))

2294


In [15]:
test = {x:{} for x in [(1,2),(3,4),(5,6)]}
test

{(1, 2): {}, (3, 4): {}, (5, 6): {}}

In [16]:
test[(1,2)] = {'a':1, 'b':2}
test[(3,4)] = {'a':3, 'b':4}
test

{(1, 2): {'a': 1, 'b': 2}, (3, 4): {'a': 3, 'b': 4}, (5, 6): {}}

In [17]:
ocurrency_example = {
        '': {'a':1, 'b':2},
        'a': {'b': 1, 'c': 1},
        ('a','b'): {'c': 1},
        ('b','c'): {'a': 2, 'd': 1},
        ('c','d'): {'a': 1, 'b': 1, 'c': 1},
}

In [5]:
def toyLM_ngram(ngram_list, ngram_sizes=[2]):
    table_list = []
    for size in ngram_sizes:
        oc_table = {}
        for i in range(len(ngram_list)-size):
            current_ngram = ngram_list[i:i+size]
            for current_ngram in ngram_list:
                if current_ngram in oc_table:
                    oc_table[current_ngram] += 1
                else:
                    oc_table[current_ngram] = 1
        table_list.append(oc_table)
        print(oc_table)
    return table_list

In [6]:
print(toyLM_ngram(texts2ids[0]))


{2: 10, 181: 10, 999: 10, 5: 10, 39: 10, 584: 10, 41: 10, 2094: 10, 2095: 10, 21: 10, 1165: 10, 530: 10}
[{2: 10, 181: 10, 999: 10, 5: 10, 39: 10, 584: 10, 41: 10, 2094: 10, 2095: 10, 21: 10, 1165: 10, 530: 10}]


In [7]:
def get_ngrams(frase, size=2):
    for i in range(size):
        frase = [0] + frase
    ngrams_list = []
    for i in range(len(frase)-size):
        ngrams_list.append((tuple(frase[i:i+size]),frase[i+size]))
    return ngrams_list

In [8]:
print(texts2ids[1])

[21, 9, 163, 246, 8, 1000, 25, 424, 1166, 5, 18, 20]


In [8]:
all_ngrams = []
for i in texts2ids:
    all_ngrams += get_ngrams(i)

In [9]:
def co_table(lista_ocurrencias):
    table = {}
    for i in lista_ocurrencias:
        if i[0] in table:
            if i[1] in table[i[0]]:
                table[i[0]][i[1]] += 1
            else:
                table[i[0]][i[1]] = 1
        else:
            table[i[0]] = {}
            table[i[0]][i[1]] = 1
    return table

In [10]:
table = co_table(all_ngrams)

In [11]:
def generate_toyLM_ngram_a(table, context='aleatorio', n=15):
    if context == 'aleatorio':
        tmp = list(table.keys())
        context = tmp[np.random.randint(len(tmp))]

    cadena = list(context)
    for _ in range(n-len(context)):
        context = tuple(cadena[-len(context):])
        if context not in table.keys():
            return tokenizer.sequences_to_texts([cadena])
        else:
            new = max(table[context], key=table[context].get)
            cadena.append(new)
    return tokenizer.sequences_to_texts([cadena])


In [23]:
generate_toyLM_ngram_a(table)

['younger generations as they are all the more we know that the commonwealth and to']

In [12]:
def generate_toyLM_ngram_b(table, context='aleatorio', n=15):
    if context == 'aleatorio':
        tmp = list(table.keys())
        context = tmp[np.random.randint(len(tmp))]

    cadena = list(context)
    for i in range(n-len(context)):
        context = tuple(cadena[-len(context):])
        if context not in table.keys():
            return tokenizer.sequences_to_texts([cadena])
        else:
            lista_tmp = list()
            for i in table[context]:
                for j in range(table[context][i]):
                    lista_tmp.append(i)
            new = np.random.randint(len(lista_tmp))
            cadena.append(lista_tmp[new])
    return tokenizer.sequences_to_texts([cadena])

In [13]:
generate_toyLM_ngram_b(table, (424, 1166))

["warm congratulations to you not from an architect's drawing board but out of date"]

# toyLM_LSTM

In [14]:
def train_generate(text, max_seq_length=10):
    train_set = dict()
    for frase in text:
        for word_index in range(len(frase)):
            if word_index < max_seq_length:
                train_set[tuple(pad_sequences([frase[:word_index]], maxlen=max_seq_length)[0])] = frase[word_index]
    return train_set
    

In [75]:
train_set = train_generate(texts2ids)
train_set
train_set = dict(list(train_set.items())[:1000])

In [78]:
x_train = to_categorical(list(train_set.keys()), num_classes=len(tokenizer.word_index))
y_train = to_categorical(list(train_set.values()), num_classes=len(tokenizer.word_index))

model = Sequential([
    Embedding(len(tokenizer.word_index), 64, input_length=10),
    LSTM(64, return_sequences=True),
    Dense(len(tokenizer.word_index), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\Peter\Desktop\Q6\PLE\ple\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Peter\Desktop\Q6\PLE\ple\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Peter\Desktop\Q6\PLE\ple\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Peter\Desktop\Q6\PLE\ple\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Peter\Desktop\Q6\PLE\ple\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Peter\Desktop\Q6\PLE\ple\lib\site-packages\keras\engine\input_spec.py", line 214, in assert_input_compatibilit

In [50]:
to_categorical(x_train, num_classes=len(tokenizer.word_index)).shape

(16062, 10, 5614)

In [52]:
y_train[0:1000].shape

(100, 5614)

In [34]:
len(tokenizer.word_index)

5614

In [40]:
len(to_categorical(list(train_set.values()), num_classes=len(tokenizer.word_index))[0])

5614